## 尝试

### task_1

In [1]:
import numpy as np
import csv
import math

def next_state(current_conf, control, dt, max_wheel_speed):
    """
    根据当前状态、控制输入和时间步长计算下一个状态。
    (此函数保持不变)
    """
    next_conf = np.zeros(12)
    phi = current_conf[0]
    saturated_wheel_speeds = np.clip(control[5:9], -max_wheel_speed, max_wheel_speed)
    delta_theta = saturated_wheel_speeds * dt
    next_conf[8:12] = current_conf[8:12] + delta_theta
    delta_J = control[0:5] * dt
    next_conf[3:8] = current_conf[3:8] + delta_J
    l, w, r = 0.47 / 2, 0.3 / 2, 0.0475
    F = (r / 4) * np.array([
        [-1/(l+w), 1/(l+w), 1/(l+w), -1/(l+w)],
        [1, 1, 1, 1],
        [-1, 1, -1, 1]
    ])
    delta_qb = F @ delta_theta
    transform_matrix = np.array([
        [1, 0, 0],
        [0, np.cos(phi), -np.sin(phi)],
        [0, np.sin(phi), np.cos(phi)]
    ])
    delta_q = transform_matrix @ delta_qb
    next_conf[0:3] = current_conf[0:3] + delta_q
    return next_conf

def run_square_pure_translation(file_name):
    """
    方法一：通过纯平移走正方形 (之前的代码)
    """
    print("--- 正在生成方法一: [纯平移] 正方形轨迹 ---")
    dt = 0.01
    side_duration = 1.0
    steps_per_side = int(side_duration / dt)
    n_steps = steps_per_side * 4
    
    u_forward = np.array([0, 0, 0, 0, 0, 10, 10, 10, 10])
    u_left = np.array([0, 0, 0, 0, 0, -10, 10, -10, 10])
    u_backward = -u_forward
    u_right = -u_left
    
    q = np.zeros(12)
    log_q = []

    for i in range(n_steps):
        if i < steps_per_side: current_u = u_forward
        elif i < 2 * steps_per_side: current_u = u_left
        elif i < 3 * steps_per_side: current_u = u_backward
        else: current_u = u_right
        
        log_q.append(np.append(q, 0))
        q = next_state(q, current_u, dt, max_wheel_speed=1000.0)

    with open(file_name, 'w', newline='') as f:
        csv.writer(f).writerows(np.array(log_q))
    
    print(f"成功生成文件: {file_name}")
    print(f"最终状态 [phi, x, y]: [{q[0]:.4f}, {q[1]:.4f}, {q[2]:.4f}]\n")

def run_square_with_rotation(file_name):
    """
    方法二：通过 前进+旋转 走正方形 (新添加的代码)
    """
    print("--- 正在生成方法二: [前进+旋转] 正方形轨迹 ---")
    dt = 0.01
    
    # 定义各阶段的持续时间
    drive_duration = 1.0  # 前进1秒
    # 根据任务3的数据计算旋转90度(pi/2)所需时间
    # 1秒旋转1.234弧度 -> x秒旋转pi/2弧度
    rotate_duration = (math.pi / 2) / 1.234 
    
    # 计算各阶段的步数
    steps_drive = int(drive_duration / dt)
    steps_rotate = int(rotate_duration / dt)

    # 定义控制输入
    u_forward = np.array([0, 0, 0, 0, 0, 10, 10, 10, 10])
    u_rotate_left = np.array([0, 0, 0, 0, 0, -10, 10, 10, -10]) # 任务3中的旋转指令

    # 计算各阶段的边界
    boundaries = [
        steps_drive,                                   # 阶段1结束
        steps_drive + steps_rotate,                    # 阶段2结束
        2 * steps_drive + steps_rotate,                # 阶段3结束
        2 * steps_drive + 2 * steps_rotate,            # 阶段4结束
        3 * steps_drive + 2 * steps_rotate,            # 阶段5结束
        3 * steps_drive + 3 * steps_rotate,            # 阶段6结束
        4 * steps_drive + 3 * steps_rotate,            # 阶段7结束
        4 * steps_drive + 4 * steps_rotate             # 阶段8结束
    ]
    n_steps = boundaries[-1]

    q = np.zeros(12)
    log_q = []

    for i in range(n_steps):
        # 根据当前步数判断处于哪个阶段，并选择控制指令
        if i < boundaries[0]: current_u = u_forward       # 1. 前进
        elif i < boundaries[1]: current_u = u_rotate_left # 2. 左转
        elif i < boundaries[2]: current_u = u_forward       # 3. 前进
        elif i < boundaries[3]: current_u = u_rotate_left # 4. 左转
        elif i < boundaries[4]: current_u = u_forward       # 5. 前进
        elif i < boundaries[5]: current_u = u_rotate_left # 6. 左转
        elif i < boundaries[6]: current_u = u_forward       # 7. 前进
        else: current_u = u_rotate_left                   # 8. 左转

        log_q.append(np.append(q, 0))
        q = next_state(q, current_u, dt, max_wheel_speed=1000.0)

    with open(file_name, 'w', newline='') as f:
        csv.writer(f).writerows(np.array(log_q))
        
    print(f"成功生成文件: {file_name}")
    print(f"总步数: {n_steps}")
    print(f"最终状态 [phi, x, y]: [{q[0]:.4f}, {q[1]:.4f}, {q[2]:.4f}]")
    print(f"最终角度 phi 约为 {q[0]:.4f} 弧度 (理论值应接近 2*pi ≈ 6.28)")

# --- 主程序入口 ---
if __name__ == "__main__":
    # 方法一：纯平移
    run_square_pure_translation("task1_square_pure_translation.csv")

    # 方法二：前进+旋转
    run_square_with_rotation("task1_square_with_rotation.csv")

--- 正在生成方法一: [纯平移] 正方形轨迹 ---
成功生成文件: task1_square_pure_translation.csv
最终状态 [phi, x, y]: [0.0000, -0.0000, -0.0000]

--- 正在生成方法二: [前进+旋转] 正方形轨迹 ---
成功生成文件: task1_square_with_rotation.csv
总步数: 908
最终状态 [phi, x, y]: [6.2675, -0.0037, 0.0037]
最终角度 phi 约为 6.2675 弧度 (理论值应接近 2*pi ≈ 6.28)


### task_2

In [2]:
import numpy as np
import csv
from scipy.linalg import expm, logm
import math # 引入math库用于pi

# --- 步骤 1: 复现 mrlib 库中的核心函数 ---

def vec_to_se3(vec):
    """将一个6维向量 (twist) 转换为一个4x4的 se(3) 矩阵。"""
    return np.array([
        [0, -vec[2], vec[1], vec[3]],
        [vec[2], 0, -vec[0], vec[4]],
        [-vec[1], vec[0], 0, vec[5]],
        [0, 0, 0, 0]
    ])

def se3_to_vec(se3_mat):
    """将一个4x4的 se(3) 矩阵转换为一个6维向量 (twist)。"""
    return np.array([
        se3_mat[2, 1], se3_mat[0, 2], se3_mat[1, 0],
        se3_mat[0, 3], se3_mat[1, 3], se3_mat[2, 3]
    ])

def matrix_exp6(twist_vec):
    """计算一个twist向量的矩阵指数。"""
    return expm(vec_to_se3(twist_vec))

def matrix_log6(T):
    """计算一个变换矩阵的矩阵对数。"""
    return se3_to_vec(logm(T))

def trans_inv(T):
    """计算一个变换矩阵的逆矩阵。"""
    R = T[0:3, 0:3]
    p = T[0:3, 3]
    R_T = R.T
    p_inv = -R_T @ p
    T_inv = np.eye(4)
    T_inv[0:3, 0:3] = R_T
    T_inv[0:3, 3] = p_inv
    return T_inv

# --- 步骤 2: 实现 ScrewTrajectory 轨迹生成函数 ---

def screw_trajectory(X_start, X_end, Tf, N, method=5):
    """
    生成一段螺旋运动轨迹。
    method=3 代表三次多项式时间缩放, method=5 代表五次多项式。
    """
    timegap = Tf / (N - 1)
    traj = []
    # 计算相对变换矩阵
    T_se_rel = trans_inv(X_start) @ X_end
    # 计算对应的twist向量
    twist_rel = matrix_log6(T_se_rel)
    
    for i in range(N):
        t = i * timegap
        if method == 5: # 五次多项式时间缩放 (更平滑)
            s = 10 * (t / Tf)**3 - 15 * (t / Tf)**4 + 6 * (t / Tf)**5
        else: # 三次多项式时间缩放
            s = 3 * (t / Tf)**2 - 2 * (t / Tf)**3
        
        # 计算当前时刻的位姿矩阵
        T_se_i = X_start @ matrix_exp6(twist_rel * s)
        traj.append(T_se_i)
        
    return traj

# --- 主轨迹生成器函数 ---

def trajectory_generator():
    """
    生成8段式抓取与放置轨迹的主函数。
    """
    # --- 步骤 3: 定义输入矩阵 ---
    Tse_initial = np.array([
        [0, 0, 1, 0.5518],
        [0, 1, 0, 0.0],
        [-1, 0, 0, 0.4012],
        [0, 0, 0, 1]
    ])
    Tsc_initial = np.array([
        [1, 0, 0, 1.0],
        [0, 1, 0, 0.0],
        [0, 0, 1, 0.025],
        [0, 0, 0, 1]
    ])
    Tsc_final = np.array([
        [0, 1, 0, 0.0],
        [-1, 0, 0, -1.0],
        [0, 0, 1, 0.025],
        [0, 0, 0, 1]
    ])
    Tce_grasp = np.array([
        [math.cos(3*math.pi/4), 0, math.sin(3*math.pi/4), 0.005],
        [0, 1, 0, 0.0],
        [-math.sin(3*math.pi/4), 0, math.cos(3*math.pi/4), -0.005],
        [0, 0, 0, 1]
    ])

    # 根据手册描述: "从standoff位置垂直往下移动0.05米可达到grasp"
    # 这意味着standoff位姿在z轴上比grasp位姿高0.05米
    Tce_standoff = Tce_grasp.copy()
    Tce_standoff[2, 3] += 0.05 # 在z方向上抬高5厘米

    # --- 步骤 4: 计算8个阶段的关键路径点(位姿矩阵) ---
    # 阶段 1 终点: 物块初始位置上方的standoff位姿
    T_se_standoff_initial = Tsc_initial @ Tce_standoff
    # 阶段 2 终点: 物块初始位置的grasp抓取位姿
    T_se_grasp_initial = Tsc_initial @ Tce_grasp
    # 阶段 5 终点: 物块目标位置上方的standoff位姿
    T_se_standoff_final = Tsc_final @ Tce_standoff
    # 阶段 6 终点: 物块目标位置的grasp放置位姿
    T_se_grasp_final = Tsc_final @ Tce_grasp

    # --- 步骤 5: 生成完整轨迹 ---
    # 仿真参数
    k = 100  # 每秒100个轨迹点
    Tf_move = 2.0  # 长距离移动段的时间
    Tf_grasp = 0.5 # 短距离抓取/释放移动段的时间
    Tf_grip = 0.5  # 爪子开合的暂停时间
    
    N_move = int(Tf_move * k)
    N_grasp = int(Tf_grasp * k)
    N_grip = int(Tf_grip * k)

    full_trajectory = []
    
    # 阶段 1: 移动到初始物块上方的standoff位置
    traj1 = screw_trajectory(Tse_initial, T_se_standoff_initial, Tf_move, N_move)
    full_trajectory.extend([(T, 0) for T in traj1]) # 爪子打开 (状态0)

    # 阶段 2: 向下移动到抓取位置
    traj2 = screw_trajectory(traj1[-1], T_se_grasp_initial, Tf_grasp, N_grasp)
    full_trajectory.extend([(T, 0) for T in traj2]) # 爪子打开 (状态0)

    # 阶段 3: 关闭爪子 (保持位置不动)
    for _ in range(N_grip):
        full_trajectory.append((traj2[-1], 1)) # 爪子关闭 (状态1)

    # 阶段 4: 向上提起物块
    traj4 = screw_trajectory(traj2[-1], T_se_standoff_initial, Tf_grasp, N_grasp)
    full_trajectory.extend([(T, 1) for T in traj4]) # 爪子关闭 (状态1)

    # 阶段 5: 将物块移动到目标位置上方的standoff位置
    traj5 = screw_trajectory(traj4[-1], T_se_standoff_final, Tf_move, N_move)
    full_trajectory.extend([(T, 1) for T in traj5]) # 爪子关闭 (状态1)

    # 阶段 6: 向下放置物块
    traj6 = screw_trajectory(traj5[-1], T_se_grasp_final, Tf_grasp, N_grasp)
    full_trajectory.extend([(T, 1) for T in traj6]) # 爪子关闭 (状态1)

    # 阶段 7: 打开爪子 (保持位置不动)
    for _ in range(N_grip):
        full_trajectory.append((traj6[-1], 0)) # 爪子打开 (状态0)

    # 阶段 8: 手臂抬起，移开
    traj8 = screw_trajectory(traj6[-1], T_se_standoff_final, Tf_grasp, N_grasp)
    full_trajectory.extend([(T, 0) for T in traj8]) # 爪子打开 (状态0)

    # --- 步骤 6: 格式化并保存为 CSV 文件 ---
    file_name = "task2_trajectory.csv"
    with open(file_name, 'w', newline='') as f:
        writer = csv.writer(f)
        for T, gripper_state in full_trajectory:
            # 根据公式 (2-6) 将4x4矩阵展平
            R = T[0:3, 0:3].flatten()
            p = T[0:3, 3]
            row = np.concatenate([R, p, [gripper_state]])
            writer.writerow(row)
            
    print(f"成功生成轨迹文件: {file_name}")
    print(f"总计轨迹点数: {len(full_trajectory)}")

# --- 运行主函数 ---
if __name__ == "__main__":
    trajectory_generator()

成功生成轨迹文件: task2_trajectory.csv
总计轨迹点数: 700
